In [ ]:

!pip install ogre-python

import matplotlib.pyplot as plt
import numpy as np

import Ogre
import Ogre.Bites
import Ogre.RTShader

class Application(Ogre.Bites.ApplicationContextBase):
    def __init__(self):
        Ogre.Bites.ApplicationContextBase.__init__(self, "OgreColab")

    def oneTimeConfig(self):
        rs = self.getRoot().getRenderSystemByName("OpenGL 3+ Rendering Subsystem")
        self.getRoot().setRenderSystem(rs)
        return True

    def pollEvents(self):
        pass


ctx = Application()
ctx.initApp()
root = ctx.getRoot()

scn_mgr = root.createSceneManager()

shadergen = Ogre.RTShader.ShaderGenerator.getSingleton()
shadergen.addSceneManager(scn_mgr)

# no RenderWindows are available here, so create an offscreen rendertarget and use that
w, h = 640, 480
tex = Ogre.TextureManager.getSingleton().createManual(
    "target", Ogre.RGN_DEFAULT, Ogre.TEX_TYPE_2D, w, h, 0, Ogre.PF_BYTE_RGB,
    Ogre.TU_RENDERTARGET, None)
win = tex.getBuffer().getRenderTarget()


In [ ]:
# this is the same code as in the numpy_sample.py example
cam = scn_mgr.createCamera("myCam")
vp = win.addViewport(cam)

# create a numpy image
arr = np.zeros((256, 256, 3), dtype=np.uint8)
arr[:, :, 1] = np.mgrid[0:256, 0:256][1]

# convert to an Ogre image and create a texture out of it
ogre_img = Ogre.Image()
ogre_img.loadDynamicImage(arr, 256, 256, Ogre.PF_BYTE_RGB)

Ogre.TextureManager.getSingleton().loadImage("gradient", "General", ogre_img)

## apply texture to a rectangle
mat = Ogre.MaterialManager.getSingleton().create("gradient_mat", "General")
rpass = mat.getTechniques()[0].getPasses()[0]
rpass.setLightingEnabled(False)
rpass.createTextureUnitState("gradient")

rect = scn_mgr.createScreenSpaceRect(True)
rect.setCorners(-0.5, 0.5, 0.5, -0.5)  # in normalized screen space
rect.setMaterial(mat)

scn_mgr.getRootSceneNode().createChildSceneNode().attachObject(rect)

vp.setBackgroundColour((0.3, 0.3, 0.3))

win.update()

## read back the pixels to a numpy array
mem = np.empty((win.getHeight(), win.getWidth(), 3), dtype=np.uint8)
pb = Ogre.PixelBox(win.getWidth(), win.getHeight(), 1, Ogre.PF_BYTE_RGB, mem)
win.copyContentsToMemory(pb, pb)

plt.imshow(mem)
